In [1]:
# Install Packages
!pip install google-api-python-client google-cloud-bigquery-storage pycountry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.7/303.7 kB 5.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 55.7 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.25.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.1.0 which is incompatible.


In [ ]:
# --- 1. SETUP AND AUTHENTICATION ---

from googleapiclient.discovery import build
from google.cloud import bigquery
from google.api_core.exceptions import NotFound
from datetime import datetime, timedelta
from datetime import datetime
import pandas as pd
import pycountry
import time
import os

# Initialize the YouTube Data API client
youtube = build("youtube", "v3", developerKey=os.getenv("YOUTUBE_API_KEY"))

print("✅ YouTube client initialized successfully")

✅ YouTube client initialized successfully


In [3]:
now = datetime.now()
year = now.year
month = now.strftime("%b").lower()  # jan, feb, mar

table_suffix = f"{year}_{month}"
table_suffix

'2026_jan'

In [4]:
# --- 2. DEFINE REQUEST PARAMETERS ---

# --- Define African regions with all 54 countries ---
regions = {
    "East Africa": ["KE", "UG", "TZ", "RW", "BI", "ET", "SO", "DJ", "ER", "SC", "MG", "MU"],
    "West Africa": ["NG", "GH", "CI", "SN", "TG", "SL", "LR", "GM", "BF", "BJ", "NE", "ML", "GN", "GW", "CV"],
    "North Africa": ["EG", "DZ", "MA", "TN", "LY", "SD"],
    "Southern Africa": ["ZA", "NA", "BW", "MZ", "ZW", "ZM", "LS", "SZ", "AO", "MW"],
    "Central Africa": ["CM", "CD", "CG", "GA", "GQ", "CF", "TD", "ST"]
}

# The API returns results in pages; this keeps track of which page to fetch next
next_page_token = None

In [5]:
# --- Initialize ---
videos = []

for region, countries in regions.items():
    print(f"Collecting data for {region} ✅")
    
    for country in countries:
        # --- Get full country name ---
        try:
            country_name = pycountry.countries.get(alpha_2=country).name
        except:
            country_name = "Unknown"
        
        try:
            next_page_token = None
            while True:
                request = youtube.videos().list(
                    part="snippet,statistics",
                    chart="mostPopular",
                    regionCode=country,
                    maxResults=50,
                    pageToken=next_page_token
                )
            
                response = request.execute()

                # --- Create a category map ---
                category_response = youtube.videoCategories().list(
                    part="snippet",
                    regionCode=country
                ).execute()
                
                category_map = {item["id"]: item["snippet"]["title"] for item in category_response.get("items", [])}

                for item in response.get("items", []):
                    snippet = item.get("snippet", {})
                    stats = item.get("statistics", {})
            
                    # Get category info
                    category_id = snippet.get("categoryId")
                    category_name = category_map.get(category_id, "Unknown")

                    videos.append({
                        "country_code": country,
                        "country_name": country_name,  # ✅ added country name
                        "region": region,
                        "video_id": item.get("id"),
                        "title": snippet.get("title"),
                        "description": snippet.get("description"),
                        "channel_title": snippet.get("channelTitle"),
                        "category_id": category_id,
                        "category_name": category_name,  # ✅ new field
                        "published_at": snippet.get("publishedAt"),
                        "tags": snippet.get("tags", []),
                        "view_count": int(stats.get("viewCount", 0)),
                        "like_count": int(stats.get("likeCount", 0)),
                        "comment_count": int(stats.get("commentCount", 0))
                    })

                next_page_token = response.get("nextPageToken")
                if not next_page_token:
                    break

        except Exception as e:
            # print(f"  ⚠️ Skipping {country} — Not supported or error occurred: {e}")
            continue

print(f"Total videos retrieved across Africa: {len(videos)} ✅")

Total videos retrieved across Africa: 2135 ✅


In [6]:
# Convert to DataFrame
bigdata = pd.DataFrame(videos)
bigdata.drop_duplicates(subset=["video_id", "country_code", "country_name", "region"], keep="first", inplace=True)
bigdata = bigdata.reset_index(drop=True)

In [7]:
bigdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2135 entries, 0 to 2134
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   country_code   2135 non-null   object
 1   country_name   2135 non-null   object
 2   region         2135 non-null   object
 3   video_id       2135 non-null   object
 4   title          2135 non-null   object
 5   description    2135 non-null   object
 6   channel_title  2135 non-null   object
 7   category_id    2135 non-null   object
 8   category_name  2135 non-null   object
 9   published_at   2135 non-null   object
 10  tags           2135 non-null   object
 11  view_count     2135 non-null   int64 
 12  like_count     2135 non-null   int64 
 13  comment_count  2135 non-null   int64 
dtypes: int64(3), object(11)
memory usage: 233.6+ KB


In [8]:
bigdata.head()

,country_code,country_name,region,video_id,title,description,channel_title,category_id,category_name,published_at,tags,view_count,like_count,comment_count
0,KE,Kenya,East Africa,ReDAxNBuZLc,D Voice feat Zuchu - Baridi (Official Visualizer),https://www.youtube.com/channel/UCxKoKHs7YwjE_...,D Voice,10,Music,2026-01-17T21:02:26Z,"[music, diamond platnumz, swahili kid, umenifu...",737810,17612,1050
1,KE,Kenya,East Africa,az-Yrg-nDEU,DEM WA SKUMA - One Woman.Two Lives ( Official ...,A fearless sukuma seller living a double life ...,KADUSKO Tv,23,Comedy,2026-01-20T11:00:05Z,"[KADUSKO Tv, KADUSKO, Comedy]",46919,345,20
2,KE,Kenya,East Africa,1A5SFR3bPfs,Senegal v Egypt | Match in 3 | CAF Africa Cup ...,Highlights from the match between Senegal and ...,SuperSport,17,Sports,2026-01-14T19:11:18Z,"[Egypt, Senegal, Senegal vs. Egypt]",497519,3668,281
3,KE,Kenya,East Africa,NjvtHjIrGTE,NDIKIAA KINDU - MAXWELL MWALIMU(Official video),#Truecammedia #Mukuniwayo #Maxwellmwalimu #mus...,MAXWELL MWALIMU Official,22,People & Blogs,2026-01-21T09:00:07Z,[],108910,6671,1470
4,KE,Kenya,East Africa,ZnfUeVeMKjY,Invincible Season 4 - Official Trailer | Prime...,March 18!!! \n\n» Watch Invincible on Prime Vi...,Prime Video,24,Entertainment,2026-01-22T17:01:02Z,"[invincible season 4 official trailer, invinci...",1229584,100420,5729


In [9]:
bigdata.tail()

,country_code,country_name,region,video_id,title,description,channel_title,category_id,category_name,published_at,tags,view_count,like_count,comment_count
2130,ZW,Zimbabwe,Southern Africa,1GiEjt2D5Fc,How i got Arrested...,I Played EVERY Fortnite Rip-Off\n𝐒𝐔𝐁𝐒𝐂𝐑𝐈𝐁𝐄 👉 ​...,Royalty Gaming,20,Gaming,2025-12-20T20:07:41Z,"[royalty gaming, royalty family gaming, ferran...",3527059,33005,4473
2131,ZW,Zimbabwe,Southern Africa,DjNKhpOC6Ag,Shone - Kuchema (Official Music Video),Video was shot in Kuwadzana Extension where Sh...,Shone,10,Music,2026-01-16T08:00:06Z,[],15905,676,69
2132,ZW,Zimbabwe,Southern Africa,9SAvVHpOrE0,AMEN - Dunsin Oyekan ft Pastor E.A Adeboye #ho...,"Even in the waiting, even in the silence, even...",Dunsin Oyekan,29,Unknown,2026-01-02T17:00:07Z,[],675741,47112,5571
2133,ZW,Zimbabwe,Southern Africa,w1JvpE2XiSo,"Tyler ICU, XDuppy, AL Xapo, Benzoo, Optimist M...",,Tyler ICU,10,Music,2025-12-24T22:00:07Z,[],1017442,19761,496
2134,ZW,Zimbabwe,Southern Africa,hbhAfcua6O4,Vybz Kartel - Victorious (Official Music Video),Vybz Kartel - Victorious (Official Music Video...,VybzKartelVEVO,10,Music,2025-12-20T17:01:13Z,"[Vybz Kartel, popcaan, onerpm, dancehall, danc...",1165790,34225,3105


In [10]:
# Initialize BigQuery client
client = bigquery.Client(project='data-storage-485106')

In [ ]:
if 'tags' in bigdata.columns:
    bigdata['tags'] = bigdata['tags'].astype(str)
else:
    bigdata['tags'] = None

In [12]:
# Define Table ID
table_id = f"data-storage-485106.youtube.trending_now_{table_suffix}"

if now.day == 1:
    try:
        prev_month_date = now.replace(day=1) - timedelta(days=1)
        prev_table_suffix = f"{prev_month_date.year}_{prev_month_date.strftime('%b').lower()}"
        prev_table_id = f"data-storage-485106.youtube.trending_now_{prev_table_suffix}"
        
        try:
            prev_data = client.query(
                f"SELECT * FROM `{prev_table_id}` ORDER BY country_code"
            ).to_dataframe()
            bigdata = pd.concat([prev_data, bigdata], ignore_index=True)
            print(f"Appended {len(prev_data)} rows from previous month table.")
        except NotFound:
            print("No previous month table found, skipping append.")
        
        job = client.load_table_from_dataframe(
            bigdata,
            table_id,
            job_config=bigquery.LoadJobConfig(write_disposition="WRITE_APPEND")
        )
        job.result()
        print(f"All data loaded into {table_id}, total rows: {len(bigdata)}")

    except Exception as e:
        print(f"Error during 1st-of-month load: {e}")

else:
    # 🔥 NORMAL WORKFLOW (this was missing)
    job = client.load_table_from_dataframe(
        bigdata,
        table_id,
        job_config=bigquery.LoadJobConfig(write_disposition="WRITE_APPEND")
    )
    job.result()
    print(f"Normal load completed into {table_id}, rows: {len(bigdata)}")

Normal load completed into data-storage-485106.youtube.trending_now_2026_jan, rows: 2135


In [13]:
# Define SQL Query to Retrieve Open Weather Data from Google Cloud BigQuery
sql = (f"""
        SELECT *
        FROM `{table_id}`
        ORDER BY country_code
       """)
    
# Run SQL Query
data = client.query(sql).to_dataframe()

In [14]:
# Delete Original Table
client.delete_table(table_id)
print(f"Table deleted successfully.")
    
# Remove Duplicate Records
data.drop_duplicates(subset=["video_id", "country_code", "country_name", "region"], inplace=True)

Table deleted successfully.


In [ ]:
if 'tags' in bigdata.columns:
    bigdata['tags'] = bigdata['tags'].astype(str)
else:
    bigdata['tags'] = None

In [16]:
# Define the dataset ID and table ID
dataset_id = 'youtube'
table_id = f"trending_now_{table_suffix}"
    
# Define the BigQuery schema for YouTube trending videos
schema = [
    bigquery.SchemaField("country_code", "STRING"),
    bigquery.SchemaField("country_name", "STRING"),
    bigquery.SchemaField("region", "STRING"),
    bigquery.SchemaField("video_id", "STRING"),
    bigquery.SchemaField("title", "STRING"),
    bigquery.SchemaField("description", "STRING"),
    bigquery.SchemaField("channel_title", "STRING"),
    bigquery.SchemaField("category_id", "STRING"),
    bigquery.SchemaField("category_name", "STRING"),
    bigquery.SchemaField("published_at", "STRING"),
    bigquery.SchemaField("tags", "STRING"),  # optional: convert list to string before upload
    bigquery.SchemaField("view_count", "INTEGER"),
    bigquery.SchemaField("like_count", "INTEGER"),
    bigquery.SchemaField("comment_count", "INTEGER")
]

# Define the table reference
table_ref = client.dataset(dataset_id).table(table_id)
    
# Create the table object
table = bigquery.Table(table_ref, schema=schema)

try:
    # Create the table in BigQuery
    table = client.create_table(table)
    print(f"Table {table.table_id} created successfully.")
except Exception as e:
    print(f"Table {table.table_id} failed")

Table trending_now_2026_jan created successfully.


In [17]:
# Define the BigQuery table ID
table_id = f"data-storage-485106.youtube.trending_now_{table_suffix}"

# Load the data into the BigQuery table
job = client.load_table_from_dataframe(data, table_id)

# Wait for the job to complete
while job.state != 'DONE':
    time.sleep(2)
    job.reload()
    print(job.state)

DONE


In [ ]:
# Define SQL Query to Retrieve Open Weather Data from Google Cloud BigQuery
sql = (f"""
        SELECT *
        FROM `{table_id}`
        ORDER BY country_code
       """)
    
# Run SQL Query
data = client.query(sql).to_dataframe()
print(f'Shape : {data.shape}')